Нужно каким-то образом загрузить данные. И попробовать нарисовать на них bouding box-ы

In [66]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import torchvision.utils as utils 

from torch.utils.data import DataLoader
from torchvision.transforms import v2

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import os
import json
import natsort

bboxes = []
# load json
with open('team_classification_data\\bboxes.json') as bboxes_json:
    bboxes = json.load(bboxes_json)

# load images 
data_dir = 'team_classification_data\\frames\\'
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == '.jpeg']
sorted_filenames = natsort.natsorted(filenames)

image_tensors = [torchvision.io.read_image(data_dir + name) for name in sorted_filenames]
batch = torch.stack(image_tensors)

for bb, fn, tens in zip(bboxes.items(), sorted_filenames, image_tensors):
    print(f'{bb[0]} - {fn}')

    team_list = []
    bboxes_list = []
    for player_id, bbox in bb[1].items():
        height = tens.shape[1]
        width = tens.shape[2]

        x = bbox['box'][0] * width
        y = bbox['box'][1] * height
        w = bbox['box'][2] * width
        h = bbox['box'][3] * height

        bboxes_list.append([x, y, w, h])
        team_list.append(int(player_id))

    bboxes_tensor = torch.tensor(bboxes_list)
    print(bboxes_tensor.shape)
    print(bboxes_tensor)
    bboxes_tensor = torchvision.ops.box_convert(bboxes_tensor[1], 'xywh', 'xyxy')

    if bb[0] > '2000': 
        img = utils.draw_bounding_boxes(tens, bboxes_tensor[1], width=2, colors='green')
        img = transforms.ToPILImage()(img)
        img.show()
        break



In [83]:
from torch.utils.data import Dataset
from PIL import Image

class TeamClassificationDataset(Dataset):
    def __init__(self, img_folder_path, bboxes_file_path, transform = None):
        super().__init__()
        self.img_folder_path = img_folder_path
        self.transform = transform

        self.bboxes = {}
        with open(bboxes_file_path) as bboxes_json:
            self.bboxes = json.load(bboxes_json)
            self.bboxes = list(self.bboxes.items())
            # print ('type(self.bboxes) = ', type(self.bboxes))
            # print(self.bboxes[0])

    def __len__(self):
        return len(self.bboxes)

    def load_image(self, imgage_path):
        image = Image.open(imgage_path).convert('RGB')
        return image

    def __getitem__(self, index):
        img_path = f'{self.img_folder_path}\\{self.bboxes[index][0]}.jpeg'
        print(f'Loading {img_path}...')
        img = self.load_image(img_path)
        if self.transform:
            img = self.transform(img)
        bbox = self.bboxes[index]
        return img, bbox


transform = v2.Compose([
                v2.PILToTensor(), # uint8 [0, 255]
                v2.ToDtype(dtype=torch.float32, scale=True), # float32, [0, 1]
                v2.Normalize((0.5,), (0.5,)), # (img - mean) / std [-1, 1]
           ])

train_dataset = TeamClassificationDataset('team_classification_data\\frames',
                                          'team_classification_data\\bboxes.json',
                                           transform)

train_dataloader = DataLoader(train_dataset)

print(len(train_dataset))

# for img, bbox in train_dataloader:
#     print(bbox)


img, bbox = next(iter(train_dataloader))
print(type(bbox[1]))


# img = torch.squeeze(img, 0)
# print(img.shape, type(img))

# plt.imshow(  img.permute(1, 2, 0),  )

# img = transforms.ToPILImage()(img)
# img.show()


100
Loading team_classification_data\frames\1310.jpeg...
<class 'list'>
